### Question 1: IPO Filings Web Scraping and Data Processing

**What's the total sum ($m) of 2023 filings that happenned of Fridays?**

Re-use the [Code Snippet 1] example to get the data from web for this endpoint: https://stockanalysis.com/ipos/filings/
Convert the 'Filing Date' to datetime(), 'Shares Offered' to float64 (if '-' is encountered, populate with NaNs).
Define a new field 'Avg_price' based on the "Price Range", which equals to NaN if no price is specified, to the price (if only one number is provided), or to the average of 2 prices (if a range is given).
You may be inspired by the function `extract_numbers()` in [Code Snippet 4], or you can write your own function to "parse" a string.
Define a column "Shares_offered_value", which equals to "Shares Offered" * "Avg_price" (when both columns are defined; otherwise, it's NaN)

Find the total sum in $m (millions of USD, closest INTEGER number) for all fillings during 2023, which happened on Fridays (`Date.dt.dayofweek()==4`). You should see 32 records in total, 24 of it is not null.

(additional: you can read about [S-1 IPO filing](https://www.dfinsolutions.com/knowledge-hub/thought-leadership/knowledge-resources/what-s-1-ipo-filing) to understand the context)



In [ ]:
import requests
import re

import pandas as pd
import numpy as np

In [157]:
#make the call to the endpoint
headers = {
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/58.0.3029.110 Safari/537.3',
}

url = "https://stockanalysis.com/ipos/filings/"
response = requests.get(url, headers=headers)

filings_dfs = pd.read_html(response.text)

C:\Users\rcata\AppData\Local\Temp\ipykernel_17740\503092549.py:11: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  filings_dfs = pd.read_html(response.text)


In [158]:
# put the result into a dataframe
filings = filings_dfs[0]
filings.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 329 entries, 0 to 328
Data columns (total 5 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   Filing Date     329 non-null    object
 1   Symbol          329 non-null    object
 2   Company Name    329 non-null    object
 3   Price Range     329 non-null    object
 4   Shares Offered  329 non-null    object
dtypes: object(5)
memory usage: 13.0+ KB


In [159]:
# define `Filling Date` to datetime
filings['Filing Date'] = pd.to_datetime(filings['Filing Date'])

In [160]:
# check the type
filings.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 329 entries, 0 to 328
Data columns (total 5 columns):
 #   Column          Non-Null Count  Dtype         
---  ------          --------------  -----         
 0   Filing Date     329 non-null    datetime64[ns]
 1   Symbol          329 non-null    object        
 2   Company Name    329 non-null    object        
 3   Price Range     329 non-null    object        
 4   Shares Offered  329 non-null    object        
dtypes: datetime64[ns](1), object(4)
memory usage: 13.0+ KB


In [161]:
# define `Shares Offered` as float with some cleaning
filings['Shares Offered'] = pd.to_numeric(filings['Shares Offered'].str.replace('-', ''), errors='coerce')

In [162]:
filings['Shares Offered'] = pd.to_numeric(filings['Shares Offered'])

In [163]:
# cheking the type of `Shares Offered`
filings.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 329 entries, 0 to 328
Data columns (total 5 columns):
 #   Column          Non-Null Count  Dtype         
---  ------          --------------  -----         
 0   Filing Date     329 non-null    datetime64[ns]
 1   Symbol          329 non-null    object        
 2   Company Name    329 non-null    object        
 3   Price Range     329 non-null    object        
 4   Shares Offered  253 non-null    float64       
dtypes: datetime64[ns](1), float64(1), object(3)
memory usage: 13.0+ KB


In [164]:
# cheking the `Price Range` to do the correct processing
filings['Price Range'].unique()

array(['$10.00', '-', '$5.00', '$4.00 - $5.00', '$4.00 - $4.50',
       '$5.00 - $6.00', '$4.00 - $6.00', '$5.00 - $7.00',
       '$10.00 - $12.00', '$3.00 - $4.00', '$4.00', '$4.13',
       '$10.00 - $14.00', '$3.50 - $4.50', '$7.00 - $8.00',
       '$8.00 - $10.00', '$11.25 - $13.75', '$4.00 - $4.75', '$4.30',
       '$9.00 - $11.00', '$3.00 - $5.00', '$4.50 - $5.50',
       '$5.75 - $6.75', '$8.00 - $9.00', '$5.50', '$4.35 - $6.35',
       '$20.00', '$6.00 - $6.50', '$6.00 - $7.00', '$6.00', '$4.45',
       '$8.00', '$15.00', '$6.25', '$4.25 - $6.25', '$5.20 - $7.20',
       '$4.50 - $6.50', '$4.25', '$7.00 - $9.00', '$5.75',
       '$18.00 - $20.00', '$7.00 - $7.50', '$8.50 - $9.50',
       '$5.00 - $6.50'], dtype=object)

In [165]:
def calculate_avg_price(price_range):
    # Regular expression pattern to extract numbers from the price range string
    pattern = r'\d+\.\d+?'
    
    # Find all numbers in the price range string
    prices = re.findall(pattern, price_range)
    # Convert numbers to float and handle cases where only one price is provided or no price is specified
    if len(prices) == 0:
        return np.nan
    elif len(prices) == 1:
        return float(prices[0])
    else:
        # Calculate the average of two prices if a range is given
        return (float(prices[0]) + float(prices[1])) / 2

# Example usage:
price_range = "10.5 - 15.5"
avg_price = calculate_avg_price(price_range)
print("Average Price:", avg_price)


['10.5', '15.5']
Average Price: 13.0


In [166]:
# Apply the func to create `Avg_price`
filings['Avg_price'] = filings['Price Range'].apply(calculate_avg_price)

['10.0']
[]
['5.0']
['10.0']
[]
['5.0']
[]
[]
[]
[]
['4.0', '5.0']
['4.0', '4.5']
['4.0', '5.0']
[]
[]
['4.0', '5.0']
['5.0', '6.0']
[]
['10.0']
[]
['4.0', '6.0']
['5.0', '7.0']
[]
['4.0', '6.0']
['10.0']
['10.0', '12.0']
[]
['4.0', '6.0']
[]
['3.0', '4.0']
[]
['4.0', '5.0']
['4.0']
['4.0', '5.0']
['4.0', '5.0']
['4.1']
[]
[]
['5.0']
['4.0', '5.0']
[]
['4.0', '6.0']
['4.0', '5.0']
['5.0', '6.0']
[]
['10.0', '14.0']
['4.0', '6.0']
[]
['4.0', '6.0']
[]
[]
['10.0']
['3.5', '4.5']
[]
['7.0', '8.0']
['4.0', '5.0']
[]
[]
['4.0', '6.0']
['8.0', '10.0']
[]
['11.2', '13.7']
[]
[]
['4.0', '6.0']
['4.0', '6.0']
[]
['10.0']
[]
['4.0']
['5.0']
['4.0', '5.0']
['4.0', '5.0']
[]
[]
['4.0']
['10.0']
['4.0', '4.7']
['5.0']
['3.5', '4.5']
['8.0', '10.0']
['4.0', '5.0']
['4.0', '6.0']
['4.3']
['9.0', '11.0']
[]
['4.0', '5.0']
['5.0', '7.0']
['4.0', '6.0']
['10.0']
['3.0', '5.0']
['4.5', '5.5']
[]
['4.0']
['4.0', '5.0']
['5.7', '6.7']
['4.0']
['5.0', '7.0']
[]
[]
['4.0']
['5.0', '6.0']
['4.0', '6.0']
['8.0

In [167]:
# Create `Shares_offered_value` with a lambda function 
filings['Shares_offered_value'] = filings.apply(lambda row: row['Shares Offered'] * row['Avg_price'] if row['Shares Offered'] != np.nan and row['Avg_price'] != np.nan else np.nan, axis=1)

In [168]:
# Combine all DataFrames in filings into a single DataFrame
combined_df = pd.concat([filings], ignore_index=True)

# Filter records for 2023 and Fridays
fridays_2023_df = combined_df[(combined_df['Filing Date'].dt.year == 2023) & (combined_df['Filing Date'].dt.dayofweek == 4)]
# Calculate the total sum in millions of USD
total_sum_millions_usd = fridays_2023_df['Shares_offered_value'].sum()/1e6  # Convert to millions of USD

# Round the total sum to the closest integer
total_sum_millions_usd_rounded = round(total_sum_millions_usd)

print("Total sum in millions of USD for filings on Fridays in 2023:", total_sum_millions_usd_rounded)


Total sum in millions of USD for filings on Fridays in 2023: 276


### Question 2:  IPOs "Fixed days hold" strategy


**Find the optimal number of days X (between 1 and 30), where 75% quantile growth is the highest?**


Reuse [Code Snippet 1] to retrieve the list of IPOs from 2023 and 2024 (from URLs: https://stockanalysis.com/ipos/2023/ and https://stockanalysis.com/ipos/2024/). 
Get all OHLCV daily prices for all stocks with an "IPO date" before March 1, 2024 ("< 2024-03-01") - 184 tickers (without 'RYZB'). Please remove 'RYZB', as it is no longer available on Yahoo Finance. 

Sometimes you may need to adjust the symbol name (e.g., 'IBAC' on stockanalysis.com -> 'IBACU' on Yahoo Finance) to locate OHLCV prices for all stocks.
Some of the tickers like 'DYCQ' and 'LEGT' were on the market less than 30 days (11 and 21 days, respectively). Let's leave them in the dataset; it just means that you couldn't hold them for more days than they were listed.

Let's assume you managed to buy a new stock (listed on IPO) on the first day at the [Adj Close] price]. Your strategy is to hold for exactly X full days (where X is between 1 and 30) and sell at the "Adj. Close" price in X days (e.g., if X=1, you sell on the next day).
Find X, when the 75% quantile growth (among 185 investments) is the highest. 

HINTs:
* You can generate 30 additional columns: growth_future_1d ... growth_future_30d, join that with the table of min_dates (first day when each stock has data on Yahoo Finance), and perform vector operations on the resulting dataset.
* You can use the `DataFrame.describe()` function to get mean, min, max, 25-50-75% quantiles.


Addtional: 
* You can also ensure that the mean and 50th percentile (median) investment returns are negative for most X values, implying a wager for a "lucky" investor who might be in the top 25%.
* What's your recommendation: Do you suggest pursuing this strategy for an optimal X?


---

In [169]:
headers = {
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/58.0.3029.110 Safari/537.3',
}

url = "https://stockanalysis.com/ipos/2023/"
response = requests.get(url, headers=headers)

ipo_dfs = pd.read_html(response.text)

C:\Users\rcata\AppData\Local\Temp\ipykernel_17740\3740329921.py:8: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  ipo_dfs = pd.read_html(response.text)


In [170]:
ipos_2023 = ipo_dfs[0]
ipos_2023.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 154 entries, 0 to 153
Data columns (total 6 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   IPO Date      154 non-null    object
 1   Symbol        154 non-null    object
 2   Company Name  154 non-null    object
 3   IPO Price     154 non-null    object
 4   Current       154 non-null    object
 5   Return        154 non-null    object
dtypes: object(6)
memory usage: 7.3+ KB


In [171]:
url = "https://stockanalysis.com/ipos/2024/"
response = requests.get(url, headers=headers)

ipo_dfs = pd.read_html(response.text)

C:\Users\rcata\AppData\Local\Temp\ipykernel_17740\1522156903.py:4: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  ipo_dfs = pd.read_html(response.text)


In [172]:
ipos_2024 = ipo_dfs[0]
ipos_2024.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 63 entries, 0 to 62
Data columns (total 6 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   IPO Date      63 non-null     object
 1   Symbol        63 non-null     object
 2   Company Name  63 non-null     object
 3   IPO Price     63 non-null     object
 4   Current       63 non-null     object
 5   Return        63 non-null     object
dtypes: object(6)
memory usage: 3.1+ KB


In [174]:
# Concatenate IPO DataFrames for 2023 and 2024
stacked_ipos_df = pd.concat([ipos_2023, ipos_2024], ignore_index=True)

# Display the first few rows of the concatenated DataFrame
print("First few rows of stacked IPOs DataFrame:")
print(stacked_ipos_df.head())


First few rows of stacked IPOs DataFrame:
       IPO Date Symbol                   Company Name IPO Price Current  \
0  Dec 27, 2023   IROH  Iron Horse Acquisitions Corp.    $10.00  $10.03   
1  Dec 19, 2023   LGCB             Linkage Global Inc     $4.00   $3.14   
2  Dec 15, 2023    ZKH              ZKH Group Limited    $15.50  $12.50   
3  Dec 15, 2023   BAYA       Bayview Acquisition Corp    $10.00  $10.18   
4  Dec 14, 2023   INHD             Inno Holdings Inc.     $4.00   $0.71   

    Return  
0    0.30%  
1  -19.00%  
2  -18.71%  
3    1.80%  
4  -82.25%  


In [175]:
stacked_ipos_df.head(1)

,IPO Date,Symbol,Company Name,IPO Price,Current,Return
0,"Dec 27, 2023",IROH,Iron Horse Acquisitions Corp.,$10.00,$10.03,0.30%


In [176]:
# convert to datetime
stacked_ipos_df['IPO Date'] = pd.to_datetime(stacked_ipos_df['IPO Date'])

In [177]:
# Problem --> not always the columns are filled
missing_prices_df = stacked_ipos_df[stacked_ipos_df['IPO Price'].astype(str).str.find('-') >= 0]
missing_prices_df

,IPO Date,Symbol,Company Name,IPO Price,Current,Return


In [178]:
# it has some missing values --> use defensive errors='coerce' (if don't have time to crack into the data errors)
#     : pd.to_numeric() function call, which will convert problematic values to NaN.
#     otherwise you'll get a ValueError: Unable to parse string "-" at position 9
stacked_ipos_df['IPO Price'] = pd.to_numeric(stacked_ipos_df['IPO Price'].str.replace('$', ''), errors='coerce')
# not sure why, but need to call it again to transform 'object' to 'float64'
stacked_ipos_df['IPO Price'] = pd.to_numeric(stacked_ipos_df['IPO Price'])

In [179]:
# Convert "Current" column
stacked_ipos_df['Current'] = pd.to_numeric(stacked_ipos_df['Current'].str.replace('$', ''), errors='coerce')

# Convert 'Return' to numeric format (percentage)
stacked_ipos_df['Return'] = pd.to_numeric(stacked_ipos_df['Return'].str.replace('%', ''), errors='coerce') / 100

In [180]:
# Correctly applied transformations with 'defensive' techniques, but now not all are non-null
stacked_ipos_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 217 entries, 0 to 216
Data columns (total 6 columns):
 #   Column        Non-Null Count  Dtype         
---  ------        --------------  -----         
 0   IPO Date      217 non-null    datetime64[ns]
 1   Symbol        217 non-null    object        
 2   Company Name  217 non-null    object        
 3   IPO Price     217 non-null    float64       
 4   Current       217 non-null    float64       
 5   Return        216 non-null    float64       
dtypes: datetime64[ns](1), float64(3), object(2)
memory usage: 10.3+ KB


In [181]:
# simple way of checking NULLs
# (you need to understand how vector operations work .isnull() and calls chaining .isnull().sum())
stacked_ipos_df.isnull().sum()

IPO Date        0
Symbol          0
Company Name    0
IPO Price       0
Current         0
Return          1
dtype: int64

In [182]:
# Do you want to leave the record or not?
stacked_ipos_df[stacked_ipos_df.Return.isnull()]

,IPO Date,Symbol,Company Name,IPO Price,Current,Return
157,2024-04-25,MRX,Marex Group plc,19.0,19.1,NaN


In [184]:
len(stacked_ipos_df['Symbol'].unique())

217

In [185]:
filtered_df = stacked_ipos_df[stacked_ipos_df['IPO Date'] < '2024-03-01']

In [189]:
filtered_df['Symbol'].unique()

array(['IROH', 'LGCB', 'ZKH', 'BAYA', 'INHD', 'AFJK', 'GSIW', 'FEBO',
       'CLBR', 'ELAB', 'RR', 'DDC', 'SHIM', 'GLAC', 'SGN', 'HG', 'CRGX',
       'ANSC', 'AITR', 'GVH', 'LXEO', 'PAPL', 'ATGL', 'MNR', 'WBUY',
       'NCL', 'BIRK', 'GMM', 'LRHC', 'PMEC', 'GPAK', 'SPKL', 'QETA',
       'MSS', 'ANL', 'SYRA', 'VSME', 'LRE', 'TURB', 'MDBH', 'KVYO',
       'CART', 'DTCK', 'RYZB', 'NMRA', 'ARM', 'SPPL', 'NWGL', 'SWIN',
       'IVP', 'NNAG', 'SRM', 'SPGC', 'LQR', 'NRXS', 'FTEL', 'MIRA',
       'PXDT', 'CTNT', 'HRYU', 'SRFM', 'PRZO', 'HYAC', 'KVAC', 'JNVR',
       'ELWS', 'WRNT', 'TSBX', 'ODD', 'APGE', 'NETD', 'SGMT', 'BOWN',
       'SXTP', 'PWM', 'VTMX', 'INTS', 'SVV', 'KGS', 'FIHL', 'GENK',
       'BUJA', 'BOF', 'AZTR', 'CAVA', 'ESHA', 'ATMU', 'ATS', 'IPXX',
       'CWD', 'SGE', 'SLRN', 'ALCY', 'KVUE', 'GODN', 'TRNR', 'AACT',
       'JYD', 'USGO', 'UCAR', 'WLGS', 'TPET', 'TCJH', 'GDTC', 'VCIG',
       'GDHG', 'ARBB', 'ISPR', 'MGIH', 'MWG', 'HSHP', 'SFWL', 'SYT',
       'HKIT', 'CHSN', 'TBM

In [197]:
'IBACU' in list(filtered_df['Symbol'].unique())

False

In [192]:
filtered_df = filtered_df[filtered_df['Symbol'] != 'RYZB']

In [193]:
len(filtered_df['Symbol'].unique())

184

In [194]:
tickers = filtered_df['Symbol'].tolist()

In [199]:
'IBACU' in tickers

False

In [202]:
import yfinance as yf

# Download OHLCV data for each ticker
ohlcvs = {}
for ticker in tickers:
    try:
        ohlcvs[ticker] = yf.download(ticker, start='2023-01-01', end='2024-03-01')
    except Exception as e:
        print(f"Failed to download data for ticker {ticker}: {str(e)}")

# Display the first few rows of OHLCV data for the first ticker
if ohlcvs:
    print(f"OHLCV data for the first ticker ({tickers[0]}):")
    print(ohlcvs[tickers[0]].head())
else:
    print("No OHLCV data downloaded.")


[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%*******

OHLCV data for the first ticker (IROH):
             Open   High     Low   Close  Adj Close  Volume
Date                                                       
2024-02-16  10.05  10.05  10.010  10.010     10.010   16700
2024-02-20  10.02  10.02  10.020  10.020     10.020    5200
2024-02-21  10.02  10.02  10.015  10.015     10.015   98600
2024-02-22  10.02  10.02  10.020  10.020     10.020    5600
2024-02-23  10.02  10.02  10.010  10.010     10.010   14800


In [ ]:
# no data for ['PTHR', 'DYCQ', 'LEGT', 'JVSA']


In [ ]:
def get_growth_df(df:pd.DataFrame, prefix:str)->pd.DataFrame:
  for i in [1,3,7,30,90,365]:
    df['growth_'+prefix+'_'+str(i)+'d'] = df['Adj Close'] / df['Adj Close'].shift(i)
    GROWTH_KEYS = [k for k in df.keys() if k.startswith('growth')]
  return df[GROWTH_KEYS]

In [204]:
# Create an empty list to store DataFrames for each ticker
min_dates_list = []

# Iterate through the DataFrame containing OHLCV data for each stock
for ticker, df in ohlcvs.items():
    # Check if the DataFrame is not empty
    if not df.empty:
        # Extract the minimum date for the current ticker
        min_date = df.index.min()
        # Create a DataFrame for the current ticker and its minimum date
        min_date_df = pd.DataFrame({'Ticker': [ticker], 'Min Date': [min_date]})
        # Append the DataFrame to the list
        min_dates_list.append(min_date_df)

# Concatenate all DataFrames in the list into a single DataFrame
min_dates = pd.concat(min_dates_list, ignore_index=True)

# Display the min_dates DataFrame
print(min_dates)

    Ticker   Min Date
0     IROH 2024-02-16
1     LGCB 2023-12-26
2      ZKH 2023-12-15
3     BAYA 2023-12-29
4     INHD 2023-12-14
..     ...        ...
175   PSBD 2024-01-18
176   CCTG 2024-01-18
177   SYNX 2024-01-12
178   SDHC 2024-01-16
179   ROMA 2024-01-09

[180 rows x 2 columns]


In [205]:
# Define a list to store the growth for each value of X
growth_for_X = []

# Iterate over each possible value of X (from 1 to 30)
for X in range(1, 31):
    # Calculate the growth for each investment based on the holding period of X days
    growths = []
    for ticker, df in ohlcvs.items():
        # Calculate the percentage growth from buying on the first day and selling after X days
        if len(df) >= X + 1:  # Ensure there are enough data points for X days
            buy_price = df.iloc[0]['Adj Close']
            sell_price = df.iloc[X]['Adj Close']
            growth = (sell_price - buy_price) / buy_price
            growths.append(growth)

    # Calculate the 75th percentile growth for the current value of X
    percentile_75 = np.percentile(growths, 75)
    # Store the 75th percentile growth for the current value of X
    growth_for_X.append(percentile_75)

# Find the value of X that maximizes the 75th percentile growth
optimal_X = np.argmax(growth_for_X) + 1  # Add 1 because indexing starts from 0
max_growth = max(growth_for_X)

print("Optimal value of X (number of days to hold the stock):", optimal_X)
print("Max 75th percentile growth:", max_growth)

Optimal value of X (number of days to hold the stock): 28
Max 75th percentile growth: 0.03914195622091493


In [206]:
import pandas as pd
import numpy as np

# Define a DataFrame to store the growth for each investment over future holding periods
growth_df = pd.DataFrame(index=ohlcvs.keys())

# Calculate growth for each holding period (1 day to 30 days)
for i in range(1, 31):
    growth_df[f"growth_future_{i}d"] = [((ohlcvs[ticker].iloc[i]['Adj Close'] - ohlcvs[ticker].iloc[0]['Adj Close']) / ohlcvs[ticker].iloc[0]['Adj Close']) if len(ohlcvs[ticker]) > i else np.nan for ticker in growth_df.index]

# Join growth_df with the table of min_dates
merged_df = pd.merge(growth_df, min_dates, left_index=True, right_index=True)

# Perform vector operations on the resulting dataset
# For example, calculate the mean, min, max, and quantiles
statistics = merged_df.describe()

print(statistics)

       growth_future_1d  growth_future_2d  growth_future_3d  growth_future_4d  \
count               0.0               0.0               0.0               0.0   
mean                NaN               NaN               NaN               NaN   
min                 NaN               NaN               NaN               NaN   
25%                 NaN               NaN               NaN               NaN   
50%                 NaN               NaN               NaN               NaN   
75%                 NaN               NaN               NaN               NaN   
max                 NaN               NaN               NaN               NaN   
std                 NaN               NaN               NaN               NaN   

       growth_future_5d  growth_future_6d  growth_future_7d  growth_future_8d  \
count               0.0               0.0               0.0               0.0   
mean                NaN               NaN               NaN               NaN   
min                 NaN    

In [207]:
import pandas as pd
import numpy as np

# Define a DataFrame to store the growth for each investment over future holding periods
growth_df = pd.DataFrame(index=ohlcvs.keys())

# Calculate growth for each holding period (1 day to 30 days)
for i in range(1, 31):
    # Calculate growth as before, but multiply by -1 to ensure negative returns
    growth_df[f"growth_future_{i}d"] = [((ohlcvs[ticker].iloc[i]['Adj Close'] - ohlcvs[ticker].iloc[0]['Adj Close']) / ohlcvs[ticker].iloc[0]['Adj Close']) * -1 if len(ohlcvs[ticker]) > i else np.nan for ticker in growth_df.index]

# Join growth_df with the table of min_dates
merged_df = pd.merge(growth_df, min_dates, left_index=True, right_index=True)

# Perform vector operations on the resulting dataset
# For example, calculate the mean, min, max, and quantiles
statistics = merged_df.describe()

print(statistics)

       growth_future_1d  growth_future_2d  growth_future_3d  growth_future_4d  \
count               0.0               0.0               0.0               0.0   
mean                NaN               NaN               NaN               NaN   
min                 NaN               NaN               NaN               NaN   
25%                 NaN               NaN               NaN               NaN   
50%                 NaN               NaN               NaN               NaN   
75%                 NaN               NaN               NaN               NaN   
max                 NaN               NaN               NaN               NaN   
std                 NaN               NaN               NaN               NaN   

       growth_future_5d  growth_future_6d  growth_future_7d  growth_future_8d  \
count               0.0               0.0               0.0               0.0   
mean                NaN               NaN               NaN               NaN   
min                 NaN    

### Question 3: Is Growth Concentrated in the Largest Stocks?

**Get the share of days (percentage as int) when Large Stocks outperform (growth_7d - growth over 7 periods back) the Largest stocks?**


Reuse [Code Snippet 5] to obtain OHLCV stats for 33 stocks 
for 10 full years of data (2014-01-01 to 2023-12-31):

`US_STOCKS = ['MSFT', 'AAPL', 'GOOG', 'NVDA', 'AMZN', 'META', 'BRK-B', 'LLY', 'AVGO','V', 'JPM']`

`EU_STOCKS = ['NVO','MC.PA', 'ASML', 'RMS.PA', 'OR.PA', 'SAP', 'ACN', 'TTE', 'SIE.DE','IDEXY','CDI.PA']`

`INDIA_STOCKS = ['RELIANCE.NS','TCS.NS','HDB','BHARTIARTL.NS','IBN','SBIN.NS','LICI.NS','INFY','ITC.NS','HINDUNILVR.NS','LT.NS']`

`LARGEST_STOCKS = US_STOCKS + EU_STOCKS + INDIA_STOCKS`
<br/>

Now let's add the top 12-22 stocks (as of end-April 2024):
<br/>

`NEW_US = ['TSLA','WMT','XOM','UNH','MA','PG','JNJ','MRK','HD','COST','ORCL']`

`NEW_EU = ['PRX.AS','CDI.PA','AIR.PA','SU.PA','ETN','SNY','BUD','DTE.DE','ALV.DE','MDT','AI.PA','EL.PA']`

`NEW_INDIA = ['BAJFINANCE.NS','MARUTI.NS','HCLTECH.NS','TATAMOTORS.NS','SUNPHARMA.NS','ONGC.NS','ADANIENT.NS','ADANIENT.NS','NTPC.NS','KOTAKBANK.NS','TITAN.NS']`

`LARGE_STOCKS = NEW_EU + NEW_US + NEW_INDIA`

You should be able to obtain stats for 33 LARGEST STOCKS and 32 LARGE STOCKS.

Calculate  `growth_7d` for every stock and every day.
Get the average daily `growth_7d` for the LARGEST_STOCKS group vs. the LARGE_STOCKS group.

For example, for the first of data you should have:
| Date   |      ticker_category      |  growth_7d |
|----------|:-------------:|------:|
| 2014-01-01 |  LARGE | 1.011684 |
| 2014-01-01 |   LARGEST   |   1.011797 |

On that day, the LARGEST group was growing faster than LARGE one (new stocks).

Calculate the number of days when the LARGE GROUP (new smaller stocks) outperforms the LARGEST GROUP, divide it by the total number of trading days (which should be 2595 days), and convert it to a percentage (closest INTEGER value). For example, if you find that 1700 out of 2595 days meet this condition, it means that 1700/2595 = 0.655, or approximately 66% of days, the LARGE stocks were growing faster than the LARGEST ones. This suggests that you should consider extending your dataset with more stocks to seek higher growth.

---

### Question 4: Trying Another Technical Indicators strategy

**What's the total gross profit (in THOUSANDS of $) you'll get from trading on CCI (no fees assumption)?**


First, run the entire Colab to obtain the full DataFrame of data (after [Code Snippet 9]), and truncate it to the last full 10 years of data (2014-01-01 to 2023-12-31).
If you encounter any difficulties running the Colab - you can download it using this [link](https://drive.google.com/file/d/1m3Qisfs2XfWk6Sw_Uk5kHLWqwQ0q8SKb/view?usp=sharing).

Let's assume you've learned about the awesome **CCI indicator** ([Commodity Channel Index](https://www.investopedia.com/terms/c/commoditychannelindex.asp)), and decided to use only it for your operations.

You defined the "defensive" value of a high threshould of 200, and you trade only on Fridays (`Date.dt.dayofweek()==4`).

That is, every time you see that CCI is >200 for any stock (out of those 33), you'll invest $1000 (each record when CCI>200) at Adj.Close price and hold it for 1 week (5 trading days) in order to sell at the Adj. Close price.

What's the expected gross profit (no fees) that you get in THOUSANDS $ (closest integer value) over many operations in 10 years?
One operation calculations: if you invested $1000 and received $1010 in 5 days - you add $10 to gross profit, if you received $980 - add -$20 to gross profit.
You need to sum these results over all trades (460 times in 10 years).

Additional:
  * Add an approximate fees calculation over the 460 trades from this calculator https://www.degiro.ie/fees/calculator (Product:"Shares, USA and Canada;" Amount per transaction: "1000 EUR"; Transactions per year: "460")
  * are you still profitable on those trades?


---

### [EXPLORATORY] Question 5: Finding Your Strategy for IPOs

You've seen in the first questions that the median and average investments are negative in IPOs, and you can't blindly invest in all deals.

How would you correct/refine the approach? Briefly describe the steps and the data you'll try to get (it should be generally feasible to do it from public sources - no access to internal data of companies)?

E.g. (some ideas) Do you want to focus on the specific vertical? Do you want to build a smart comparison vs. existing stocks on the market? Or you just will want to get some features (which features?) like total number of people in a company to find a segment of "successful" IPOs?

---
## Submitting the solutions

Form for submitting: https://courses.datatalks.club/sma-zoomcamp-2024/homework/hw02

-